<a href="https://colab.research.google.com/github/AnnaBo1/python_for_DA/blob/main/%22hw_6_4_%D0%90%D0%BD%D0%B0%D0%BB%D1%96%D0%B7_%D0%90%D0%92_%D1%82%D0%B5%D1%81%D1%82%D1%83_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Аналіз A/B тесту

Маємо проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це класична гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку. У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми проаналізуємо вплив на утримання (retention) гравців. Тобто хочемо зрозуміти чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Змінні в даних наступні:

- userid - унікальний номер, який ідентифікує кожного гравця.
- version - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- sum_gamerounds - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- retention_1 - чи через 1 день після встановлення гравець повернувся і почав грати?
- retention_7 - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [1]:
import pandas as pd

# Зчитуємо дані з CSV
df = pd.read_csv('cookie_cats.csv')

# Виведемо перші кілька рядків, щоб побачити структуру даних
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [2]:
# Розраховуємо середнє значення retention_7 для обох версій гри
mean_retention_7 = df.groupby('version')['retention_7'].mean()


# Виводимо середні значення утримання на 7 день для обох груп
print(mean_retention_7)

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64


gate_30 (контрольна група): 19.02%
gate_40 (тестова група): 18.20%

На основі простого порівняння середніх значень можна побачити, що гравці, які знаходилися у версії гри з воротами на рівні 30, мають трохи краще утримання через 7 днів (19.02%) порівняно з тими, хто грав у версії з воротами на рівні 40 (18.20%).

    Формулювання гіпотези:

Нульова гіпотеза (H0): Середнє утримання на 7-й день однакове для обох версій гри.
Альтернативна гіпотеза (H1): Є різниця в утриманні між версіями гри.

2. Перевірте з допомогою z-тесту аналогічно до прикладу в лекції, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для двох вибірок. Виведіть результат у форматі:
```
z statistic: ...
p-value: ...
Довірчий інтервал 95% для групи control: [..., ...]
Довірчий інтервал 95% для групи treatment: [..., ...]
```
де замість `...` - обчислені значення. В якості висновка дайте відповідь на два питання:  
    1. чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
    2. чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  
    
Зверніть увагу, в такому і схожому завданнях ми використовуєм `proportion` Z-тест. Це тому що в нас залежна змінна має бінарне значення (повернеться аби ні користувач, чи клікне або ні користувач в інших ситуаціях - всього два можливих значення в змінної: 0/1, True/False ). Якщо б ми вимірювали скажімо чи є стат. значущою різниця між вагою чоловіків і жінок в певній вибірці, ми б використовувавли функцію `statsmodels.stats.ztest`, бо залежна змінна `вага` є неперервною (тип float, замість типу int чи bool і тільки двох можливих значень).

In [3]:
# Спочатку фільтруємо дані для контрольної і тестової груп
control_results = df[df['version'] == 'gate_30']['retention_7']
treatment_results = df[df['version'] == 'gate_40']['retention_7']

In [4]:
# Підрахунок кількості гравців та кількості успішних повернень
n_con = control_results.count()
n_treat = treatment_results.count()

In [5]:
# Підрахунок успіхів (повернень) для кожної групи
successes = [control_results.sum(), treatment_results.sum()]

In [6]:
# Загальна кількість спостережень у кожній групі
nobs = [n_con, n_treat]

In [7]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

# Z-тест для двох пропорцій
z_stat, p_value = proportions_ztest(successes, nobs)

In [8]:
# Довірчі інтервали для контрольної групи
control = proportion_confint(successes[0], nobs[0], alpha=0.05)

In [9]:
# Довірчі інтервали для тестової групи
treatment = proportion_confint(successes[1], nobs[1], alpha=0.05)

In [10]:
# Виведення результатів
print(f"Z-statistic: {z_stat:.3f}")
print(f"P-value: {p_value:.3f}")
print(f"Довірчий інтервал 95% для контрольної групи: {control}")
print(f"Довірчий інтервал 95% для тестової групи: {treatment}")

Z-statistic: 3.164
P-value: 0.002
Довірчий інтервал 95% для контрольної групи: (0.18656311652199903, 0.19383956804175934)
Довірчий інтервал 95% для тестової групи: (0.17845430073314686, 0.18554578720019968)


    1.чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?
    Статистична значущість
Z-statistic: 3.164

P-value: 0.002

P-value менше 0.05 (а в даному випадку — 0.002), що свідчить про статистично значущу різницю між поведінкою користувачів у контрольній та тестовій групах. Це означає, що зміни, які були внесені в тестову групу , вплинули на поведінку користувачів.

    2.чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?
Довірчий інтервал 95% для контрольної групи: (0.1865, 0.1938)

Довірчий інтервал 95% для тестової групи: (0.1784, 0.1855)

Не перетинаються довірчі інтервали. Це також вказує на те, що існує значна різниця в середніх значеннях утримання між контрольними і тестовими групами.

3. Є ще один тип тестів, який використовується для бінарної метрики як от "зробить юзер дію, чи ні" - тест [**Хі-квадрат**](https://www.bmj.com/about-bmj/resources-readers/publications/statistics-square-one/8-chi-squared-tests) (ще одне [пояснення](https://www.scribbr.com/statistics/chi-square-tests/) тесту з прикладами). В нього інші гіпотези Н0 і Н1 на відміну від z- та t-тестів. А також цей тест можна використовувати, якщо в нас більше за 2 досліджувані групи, тобто в нас не А/В тест, а А/B/C/D, наприклад.  

В **z- та t-тестах** (які відрізняються тим, що ми в першому не знаємо дисперсію генеральної сукупності, але якщо в нас великий набір даних, то ці два тести дають дуже схожі результати) **ми перевіряємо, чи є різниця у середніх показниках по групам користувачів**.  

А в **тесті Хі-квадрат ми перевіряємо чи є звʼязок між групою користувача і тим, чи він зробить цікаву нам дію**. Це ніби дослідження одного і того самого, але дещо різними способами. Для перевірки, можна виконувати кілька тестів (особливо, якщо один дає якийсь непереконливий результат типу р-значення 0.07 - наче і fail to regect H0 на рівні стат значущості 5%, але цікаво, що скажуть інші тести), тож, зробимо і ми тест хі-квадрат та порівняємо його результат з z-тестом.

Про різницю між тестами можна почитати ще [тут](https://stats.stackexchange.com/a/178860) - це просто пояснення користувача стековерфлоу, але там розумні люди сидять.

Для проведення хі-квадрат тесту скористаємось функцією з `scipy.stats` `chi2_contingency` для обчислення статистики хі-квадрат і р-значення для перевірки конкретної гіпотези. У цю функцію вам треба передати таблицю 2х2: кількість випадків для кожної версії гри і значення `retention_7`.

**Задача**: виконайте тест хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та тим, чи зайде гравець на 7ий день після встановлення гри.
Тут гіпотези наступні
- Н0: значення retention_7 не залежить від версії гри
- Н1: є залежність між версією гри і значенням retention_7

Виведіть p-значення та зробіть висновок.


In [13]:
#Для проведення хі-квадрат тесту скористаємось функцією з
from scipy.stats import chi2_contingency

successes[0] - кількість гравців, які повернулися на 7-й день у контрольній групі.

n_con - загальна кількість гравців у контрольній групі.

successes[1] - кількість гравців, які повернулися на 7-й день у тестовій групі.

n_treat - загальна кількість гравців у тестовій групі.

In [14]:
# Створюємо таблицю 2x2
table = [
    [successes[0], n_con - successes[0]],  # Версія A (контрольна)
    [successes[1], n_treat - successes[1]]  # Версія B (тестова)
]

In [15]:
# Виконуємо хі-квадрат тест
chi2_stat, p_value, dof, expected = chi2_contingency(table)

In [16]:
print(p_value)

0.0016005742679058301


In [17]:
if p_value < 0.05:
    print("Є статистично значуща залежність між версією гри та утриманням на 7-й день.")
else:
    print("Немає статистично значущої залежності між версією гри та утриманням на 7-й день.")

Є статистично значуща залежність між версією гри та утриманням на 7-й день.


p_value менше 0.05 i ми можемо стверджувати, що існує залежність між версією гри та показником утримання на 7-й день